# Lab | Data Structuring and Combining Data

## Challenge 1: Combining & Cleaning Data

In this challenge, we will be working with the customer data from an insurance company, as we did in the two previous labs. The data can be found here:
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv

But this time, we got new data, which can be found in the following 2 CSV files located at the links below.

- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv
- https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv

Note that you'll need to clean and format the new data.

Observation:
- One option is to first combine the three datasets and then apply the cleaning function to the new combined dataset
- Another option would be to read the clean file you saved in the previous lab, and just clean the two new files and concatenate the three clean datasets

In [227]:
# import libraries
import pandas as pd
import re 

# Load the data
client_data = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file1.csv")
client_data2 = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file2.csv")
client_data3 = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/file3.csv")

In [228]:
# Explore the df

client_data.head()
client_data2.head()
client_data3.head()



,Customer,State,Customer Lifetime Value,Education,Gender,Income,Monthly Premium Auto,Number of Open Complaints,Policy Type,Total Claim Amount,Vehicle Class
0,SA25987,Washington,3479.137523,High School or Below,M,0,104,0,Personal Auto,499.200000,Two-Door Car
1,TB86706,Arizona,2502.637401,Master,M,0,66,0,Personal Auto,3.468912,Two-Door Car
2,ZL73902,Nevada,3265.156348,Bachelor,F,25820,82,0,Personal Auto,393.600000,Four-Door Car
3,KX23516,California,4455.843406,High School or Below,F,0,121,0,Personal Auto,699.615192,SUV
4,FN77294,California,7704.958480,High School or Below,M,30366,101,2,Personal Auto,484.800000,SUV


In [229]:
# Clean column names. 
# Al the sets seem to contain the same kind of data, but the columns names are a bit differentWe need to standardise them
for df in [client_data, client_data2, client_data3]: 
    df.columns = df.columns.str.strip().str.replace("ST", "state").str.lower().str.replace(' ', '_')

print(client_data.columns)
print(client_data2.columns)
print(client_data3.columns)

Index(['customer', 'state', 'gender', 'education', 'customer_lifetime_value',
       'income', 'monthly_premium_auto', 'number_of_open_complaints',
       'policy_type', 'vehicle_class', 'total_claim_amount'],
      dtype='object')
Index(['customer', 'state', 'gender', 'education', 'customer_lifetime_value',
       'income', 'monthly_premium_auto', 'number_of_open_complaints',
       'total_claim_amount', 'policy_type', 'vehicle_class'],
      dtype='object')
Index(['customer', 'state', 'customer_lifetime_value', 'education', 'gender',
       'income', 'monthly_premium_auto', 'number_of_open_complaints',
       'policy_type', 'total_claim_amount', 'vehicle_class'],
      dtype='object')


In [230]:

# Combine the dataframes
# We can now concatenate the dataframes together

client_data_123 = pd.concat([client_data, client_data2, client_data3], ignore_index=True)

In [231]:
# Clean the data
# Start by exploring the data
client_data_123.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12074 entries, 0 to 12073
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customer                   9137 non-null   object 
 1   state                      9137 non-null   object 
 2   gender                     9015 non-null   object 
 3   education                  9137 non-null   object 
 4   customer_lifetime_value    9130 non-null   object 
 5   income                     9137 non-null   float64
 6   monthly_premium_auto       9137 non-null   float64
 7   number_of_open_complaints  9137 non-null   object 
 8   policy_type                9137 non-null   object 
 9   vehicle_class              9137 non-null   object 
 10  total_claim_amount         9137 non-null   float64
dtypes: float64(3), object(8)
memory usage: 1.0+ MB


In [232]:

# 1 Remove empty rows. 
# We can use the `dropna()` function in pandas to remove null values
client_data_123 = client_data_123.dropna(how="all")

# We can use the `isnull()` function to check for remaining null values
# only gender and customer_lifetime_value have null values. 
client_data_123.isnull().sum()


customer                       0
state                          0
gender                       122
education                      0
customer_lifetime_value        7
income                         0
monthly_premium_auto           0
number_of_open_complaints      0
policy_type                    0
vehicle_class                  0
total_claim_amount             0
dtype: int64

In [233]:
# 2 Cleaning invalid values and correct data types: 

# 2.1 state: We can use the `replace()` function to replace the values
client_data_123.state=client_data_123.state.replace({"WA": "Washington", "AZ":"Arizona", "Cali": "California"})
client_data_123.state.unique()

# 2.2 gender: Replace invalid values and fill in the missing values:
client_data_123.gender = client_data_123.gender.apply(lambda x: x[0].upper() if isinstance(x, str) else "unspecified")
client_data_123.gender.unique()

# 2.3 education: Replace invalid values
client_data_123.education = client_data_123.education.replace({"Bachelors": "Bachelor"})
client_data_123.education.unique()

# 2.4 customer_lifetime_value: Correct data type and fill in null values with the median value. 
client_data_123.customer_lifetime_value = pd.to_numeric(client_data_123.customer_lifetime_value.str.replace("%", ""), errors="coerce")
client_data_123.customer_lifetime_value = client_data_123.customer_lifetime_value.fillna(client_data_123.customer_lifetime_value.median())

# 2.5 income: nothing to clean on first sight

# 2.6 monthly_premium_auto: nothing to clean on first sight

# 2.7 number_of_open_complaints
client_data_123.number_of_open_complaints = pd.to_numeric(client_data_123.number_of_open_complaints.apply(lambda x: x.split("/")[1] if isinstance(x, str) else x), errors="coerce")

# 2.9 policy_type: nothing to change

# 2.10 vehicle_class: update classes to grupo some under "luxury"
client_data_123.vehicle_class = client_data_123.vehicle_class.replace({"Sports Car": "Luxury","Luxury SUV":"Luxury","Luxury Car":"Luxury"})


# 2.11 total_claim_amount: nothing to change


In [234]:
client_data_123.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9137 entries, 0 to 12073
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customer                   9137 non-null   object 
 1   state                      9137 non-null   object 
 2   gender                     9137 non-null   object 
 3   education                  9137 non-null   object 
 4   customer_lifetime_value    9137 non-null   float64
 5   income                     9137 non-null   float64
 6   monthly_premium_auto       9137 non-null   float64
 7   number_of_open_complaints  9137 non-null   int64  
 8   policy_type                9137 non-null   object 
 9   vehicle_class              9137 non-null   object 
 10  total_claim_amount         9137 non-null   float64
dtypes: float64(4), int64(1), object(6)
memory usage: 856.6+ KB


# Challenge 2: Structuring Data

In this challenge, we will continue to work with customer data from an insurance company, but we will use a dataset with more columns, called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by performing data cleaning, formatting, and structuring.

In [235]:
# Load data from url
marketing_customer_analysis = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv")

In [236]:
# Explore the data to understand it and clean it
marketing_customer_analysis.info()

for column in marketing_customer_analysis.columns: 
    print(column, marketing_customer_analysis[column].unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10910 entries, 0 to 10909
Data columns (total 27 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   unnamed:_0                     10910 non-null  int64  
 1   customer                       10910 non-null  object 
 2   state                          10910 non-null  object 
 3   customer_lifetime_value        10910 non-null  float64
 4   response                       10910 non-null  object 
 5   coverage                       10910 non-null  object 
 6   education                      10910 non-null  object 
 7   effective_to_date              10910 non-null  object 
 8   employmentstatus               10910 non-null  object 
 9   gender                         10910 non-null  object 
 10  income                         10910 non-null  int64  
 11  location_code                  10910 non-null  object 
 12  marital_status                 10910 non-null 

In [237]:
# Everything seems correct except for:
# * the unnamed:_0 column which looks like an old index and should be removed. 
marketing_customer_analysis = marketing_customer_analysis.drop(columns=["unnamed:_0"])
# * one of the column names (employmentstatus)
marketing_customer_analysis.rename(columns={"employmentstatus":"employment_status"}, inplace=True)
# * number_of_open_complaints which should be an int 
marketing_customer_analysis.number_of_open_complaints = marketing_customer_analysis.number_of_open_complaints.apply(lambda x: round(x)).astype(int)
# * months_since_last_claim which should be an int.
marketing_customer_analysis.months_since_last_claim = marketing_customer_analysis.months_since_last_claim.apply(lambda x: round(x)).astype(int)
# * total_claim_amount could be standardised to 2 decimal places. 
marketing_customer_analysis.total_claim_amount = marketing_customer_analysis.total_claim_amount.apply(lambda x: round(x, 2))

1. You work at the marketing department and you want to know which sales channel brought the most sales in terms of total revenue. Using pivot, create a summary table showing the total revenue for each sales channel (branch, call center, web, and mail).
Round the total revenue to 2 decimal points.  Analyze the resulting table to draw insights.

In [238]:
# total revenue for each sales channel. 

marketing_customer_analysis.groupby("sales_channel").monthly_premium_auto.sum().astype(float).apply(lambda x: round(x,2))

pivot_revenue = marketing_customer_analysis.pivot_table(columns='sales_channel', values=['monthly_premium_auto'], aggfunc='sum')
pivot_revenue

sales_channel,Agent,Branch,Call Center,Web
monthly_premium_auto,386335,280953,197970,151511


2. Create a pivot table that shows the average customer lifetime value per gender and education level. Analyze the resulting table to draw insights.

In [240]:
pivot_lifetime_value = marketing_customer_analysis.pivot_table(index='gender', columns='education', values=['customer_lifetime_value'], aggfunc='mean')
pivot_lifetime_value

customer_lifetime_value                            \
education                Bachelor      College       Doctor   
gender                                                        
F                     7874.269478  7748.823325  7328.508916   
M                     7703.601675  8052.459288  7415.333638   

                                             
education High School or Below       Master  
gender                                       
F                  8675.220201  8157.053154  
M                  8149.687783  8168.832659

## Bonus

You work at the customer service department and you want to know which months had the highest number of complaints by policy type category. Create a summary table showing the number of complaints by policy type and month.
Show it in a long format table.

*In data analysis, a long format table is a way of structuring data in which each observation or measurement is stored in a separate row of the table. The key characteristic of a long format table is that each column represents a single variable, and each row represents a single observation of that variable.*

*More information about long and wide format tables here: https://www.statology.org/long-vs-wide-data/*

In [152]:
# Your code goes here